In [1]:
# Importing the libraries here

import pandas as pd
import numpy as np
import ast
import json
from operator import add 
from operator import truediv
import re
# import swifter
from tqdm import tqdm

In [2]:
#data with TECH removed or non master removed and tech removed
# summary = pd.read_csv("BERT_Pred_189_FULL_with_eng.csv")
# summary = pd.read_csv("BERT_Pred_189_FULL_without_MASTER.csv")
summary = pd.read_csv("Bert_Preds_EXP_Summary_ENG_only.csv")
# summary = pd.read_csv("BERT_Pred_500_without_MASTER_ENG+TECH_ROW_WISE.csv")

In [3]:
summary.head()

,id,summaries,manually Tagged,BERT_Tags
0,-4W7H0UFJneCeHmDK-b9LA_0000,im a seasoned software consultant and manager...,['gmail'],[]
1,-5KQNWRmZUAxcmAKsBVFCw_0000,committed and clientfocused technical lead wi...,"['oracle pl/sql', 'vb.net', 'hp quality center...",['tidal']
2,-9vRkMCaww4q9MTuMhg16A_0000,professional web site design and development ...,"['html', 'php', 'javascript']",['html']
3,-nZOV-0MncypcjMiVI38pQ_0000,experienced it software and infrastructure pr...,"['versionone', 'ios', 'sharepoint', 'windows 1...",['dell']
4,0-6kc4T3YjlcPQYOK7Wt4g_0000,now hiring actively sourcing nfoh manager agm...,['gmail'],[]


In [4]:
summary.columns = ['id','summaries','exact_matched_patt_contextual','BERT_Tags']

In [5]:
dataa = summary[['id','summaries','exact_matched_patt_contextual','BERT_Tags']]

# dataa = summary[['ID','summaries','exact_matched_patt_contextual','string_matched']]

In [6]:
summary=dataa

In [7]:
summary = summary.sort_values(by=['id'], ascending=False)
summary.reset_index(inplace = True, drop = True)

In [8]:
col_name_op = "BERT_Tags"
# col_name_op = "string_matched"

In [9]:
# Pass the matched technology column that we got
def convert_data_format(df,column_name):
    
    for i in range(0, len(df)):
        
        if df[column_name][i][0] == '[':
            res = ast.literal_eval(df[column_name][i])
            df[column_name][i] = res
            
        else:
            x = []
            x.insert(0, df[column_name][i])
            df[column_name][i] = x
            
    return df

df_cm = summary

In [10]:
df_cm = convert_data_format(df_cm, col_name_op)

In [11]:
df_cm = convert_data_format(df_cm, "exact_matched_patt_contextual")

In [12]:
df_cm[col_name_op][12]

[]

In [13]:
df_cm["exact_matched_patt_contextual"][12]

['gmail']

In [14]:
count_context_technologies = []

for i in range(0, len(df_cm)):
    count_context_technologies.extend(df_cm["exact_matched_patt_contextual"][i])
d = list(set(count_context_technologies))
len(d)

362

# Converting into dataformat to compare two strings

In [15]:
# TP: present in both sheet i==j
# TN: not in both (insignificant)
# FP: in extracted sheet but not in manual tagging     i!=j
# FN: in manual tagging and is not in extracted tagging j

In [16]:
tp = []
tn = []
fp = []
fn = []

In [17]:
TP = TN = FP = FN = 0

# TP + FP calculation
# op_string = exact_matched_patt
# , manual_tagged_string = Actual/Actual(NER)

df_cm.head()

,id,summaries,exact_matched_patt_contextual,BERT_Tags
0,zeExhAA-r0lbcTQH41BlFg_0000,senior account manager at verizon enterprise ...,[verizon enterprise],[]
1,zSvilx9nLA2ADlgkbh0Avw_0000,12 years of software development experience o...,"[microsoft.net, asp.net, java, jquery, html, j...","[html, java, angular]"
2,z5Kl-V2w51Z-6uhXbNCRrQ_0000,ours excavating is a commercial excavating de...,[gmail],[]
3,yCI14syBYDiD3pwIDKVEKQ_0000,getting things done voor iedere uitdaging de ...,[gmail],[]
4,xuMF3K3aFCBGMOElWU0WTA_0000,12 +years of experience in development qa dev...,"[java, selenium]","[eclipse, java, selenium]"


In [18]:
df_cm["exact_matched_patt_contextual"] = df_cm["exact_matched_patt_contextual"].fillna('[]')

# reading 
tech = pd.read_csv("/Users/mohitbagaria/Downloads/spacy_string_match/Eng_pydictionary_2.csv")
# tech = pd.read_csv("Tech_pydictionary_2.csv")

tech.columns = ["index", "title"]
tech.drop(columns = ["index"], axis =1 , inplace = True)
tech.reset_index(inplace = True, drop = True)

In [19]:
tech.head()

,title
0,Lattice
1,EMERGE Background Checks
2,Tenfold
3,Friday Feedback
4,Data Dwell


In [20]:
tech_list = tech["title"].tolist()
tech_list = list(set(tech_list))
len(tech_list)

5289

In [21]:
for i in range(0, len(tech_list)):
    tech_list[i] = tech_list[i].lower()

In [22]:
# tech_list

In [28]:
df_cm['Eng_Label']=df_cm["exact_matched_patt_contextual"].copy()

In [29]:
tech_tag_list = []

def get_tech_tagged_data(manually_tagged, tech_list):
    keep=[]
    for word in manually_tagged:
        if word in tech_list:
            keep.append(word)
    return keep

In [30]:
df_cm.head(20)

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label
0,zeExhAA-r0lbcTQH41BlFg_0000,senior account manager at verizon enterprise ...,[verizon enterprise],[],[verizon enterprise]
1,zSvilx9nLA2ADlgkbh0Avw_0000,12 years of software development experience o...,"[microsoft.net, asp.net, java, jquery, html, j...","[html, java, angular]","[microsoft.net, asp.net, java, jquery, html, j..."
2,z5Kl-V2w51Z-6uhXbNCRrQ_0000,ours excavating is a commercial excavating de...,[gmail],[],[gmail]
3,yCI14syBYDiD3pwIDKVEKQ_0000,getting things done voor iedere uitdaging de ...,[gmail],[],[gmail]
4,xuMF3K3aFCBGMOElWU0WTA_0000,12 +years of experience in development qa dev...,"[java, selenium]","[eclipse, java, selenium]","[java, selenium]"
5,xlZw0zgxhTJ34Zkj8qz3ow_0000,i am an upcoming graduate from the university...,[gmail],[],[gmail]
6,xi1WETjz2xTW4jBSE8U8DQ_0000,specialties advertising technology agile soft...,"[java, javascript]",[java],"[java, javascript]"
7,xfwmFMDDgwYyhZMc1MJcpg_0000,cpa eligible currently taking cpa exams and a...,[gmail],[],[gmail]
8,wy3R8tztPGIHWzOl1wTTOg_0000,an experienced test consultant in the field o...,"[ibm rational, soapui, selenium, javascript, j...","[python, selenium, java]","[ibm rational, soapui, selenium, javascript, j..."
9,wmiO3uJcgJcjGMruOdWykg_0000,an accomplished resultsoriented customer supp...,[gmail],[],[gmail]


In [31]:
# df_cm['Tech_Label'] = df_cm.apply(lambda x: get_tech_tagged_data(x["MANUAL_TAG"], tech_list), axis = 1)
df_cm['Eng_Label'] = df_cm.apply(lambda x: get_tech_tagged_data(x["Eng_Label"], tech_list), axis = 1)

In [32]:
df_cm.head(20)

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label
0,zeExhAA-r0lbcTQH41BlFg_0000,senior account manager at verizon enterprise ...,[verizon enterprise],[],[]
1,zSvilx9nLA2ADlgkbh0Avw_0000,12 years of software development experience o...,"[microsoft.net, asp.net, java, jquery, html, j...","[html, java, angular]","[java, html, angular]"
2,z5Kl-V2w51Z-6uhXbNCRrQ_0000,ours excavating is a commercial excavating de...,[gmail],[],[]
3,yCI14syBYDiD3pwIDKVEKQ_0000,getting things done voor iedere uitdaging de ...,[gmail],[],[]
4,xuMF3K3aFCBGMOElWU0WTA_0000,12 +years of experience in development qa dev...,"[java, selenium]","[eclipse, java, selenium]","[java, selenium]"
5,xlZw0zgxhTJ34Zkj8qz3ow_0000,i am an upcoming graduate from the university...,[gmail],[],[]
6,xi1WETjz2xTW4jBSE8U8DQ_0000,specialties advertising technology agile soft...,"[java, javascript]",[java],[java]
7,xfwmFMDDgwYyhZMc1MJcpg_0000,cpa eligible currently taking cpa exams and a...,[gmail],[],[]
8,wy3R8tztPGIHWzOl1wTTOg_0000,an experienced test consultant in the field o...,"[ibm rational, soapui, selenium, javascript, j...","[python, selenium, java]","[selenium, java, python]"
9,wmiO3uJcgJcjGMruOdWykg_0000,an accomplished resultsoriented customer supp...,[gmail],[],[]


In [ ]:
# df_cm.to_csv("technology_dataset_with_taggging.csv", index= False)
# df_cm.to_csv("non_technology_dataset_with_taggging.csv", index= False)

In [33]:
list_tech_dataset = []

In [34]:
# fill with 
for i in range(0, len(df_cm)):
    list_tech_dataset.extend(df_cm["Eng_Label"][i])

In [35]:
len(list_tech_dataset)

318

In [36]:
set_tech = set(list_tech_dataset)
len(set_tech)

108

In [37]:
for i in range(0, len(df_cm)):
    l = df_cm[col_name_op][i]
    df_cm[col_name_op][i] = [0 if element == '0' else element for element in l]

In [38]:
for i in range(0, len(df_cm)):
  words = df_cm[col_name_op][i]
  stopwords = [0]
  for word in list(words):
    if word in stopwords:
      words.remove(word)

In [39]:
def Diff(li1, li2): #FP entries
    return list(set(li2)-set(li1))

In [40]:
def Same(lis1,lis2): #TP entries
    return list(set(lis1).intersection(lis2))

In [41]:
def Common(lis1,lis2):
    return list(set(lis1)-(set(lis2)))

In [42]:
# list of technologies which are not in extracted list but are in tagged list i.e. FN entries
df_cm['FP_entries_eng'] = df_cm.apply(lambda x: Diff(x["Eng_Label"], x[col_name_op]), axis = 1)

In [43]:
# list of technologies which are in extracted list and are in tagged list i.e. TP entries
df_cm['TP_entries_eng'] = df_cm.apply(lambda x: Same(x["Eng_Label"], x[col_name_op]), axis = 1)

In [44]:
# list of technologies which are NOT extracted list and are in tagged list i.e. FN entries
df_cm['FN_entries_eng'] = df_cm.apply(lambda x: Common(x["Eng_Label"], x[col_name_op]), axis = 1)

In [45]:
df_cm.head(15)

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label,FP_entries_eng,TP_entries_eng,FN_entries_eng
0,zeExhAA-r0lbcTQH41BlFg_0000,senior account manager at verizon enterprise ...,[verizon enterprise],[],[],[],[],[]
1,zSvilx9nLA2ADlgkbh0Avw_0000,12 years of software development experience o...,"[microsoft.net, asp.net, java, jquery, html, j...","[html, java, angular]","[java, html, angular]",[],"[html, angular, java]",[]
2,z5Kl-V2w51Z-6uhXbNCRrQ_0000,ours excavating is a commercial excavating de...,[gmail],[],[],[],[],[]
3,yCI14syBYDiD3pwIDKVEKQ_0000,getting things done voor iedere uitdaging de ...,[gmail],[],[],[],[],[]
4,xuMF3K3aFCBGMOElWU0WTA_0000,12 +years of experience in development qa dev...,"[java, selenium]","[eclipse, java, selenium]","[java, selenium]",[eclipse],"[java, selenium]",[]
5,xlZw0zgxhTJ34Zkj8qz3ow_0000,i am an upcoming graduate from the university...,[gmail],[],[],[],[],[]
6,xi1WETjz2xTW4jBSE8U8DQ_0000,specialties advertising technology agile soft...,"[java, javascript]",[java],[java],[],[java],[]
7,xfwmFMDDgwYyhZMc1MJcpg_0000,cpa eligible currently taking cpa exams and a...,[gmail],[],[],[],[],[]
8,wy3R8tztPGIHWzOl1wTTOg_0000,an experienced test consultant in the field o...,"[ibm rational, soapui, selenium, javascript, j...","[python, selenium, java]","[selenium, java, python]",[],"[python, java, selenium]",[]
9,wmiO3uJcgJcjGMruOdWykg_0000,an accomplished resultsoriented customer supp...,[gmail],[],[],[],[],[]


In [46]:
# df_cm.to_csv("bert_eng_accuracy_M1_with_entries.csv", index = False)

In [47]:
def get_TP(df,manual_tagged_string,op_string):
    
    for ind in df.index:
        TP = FP = 0
        
        for i in range(0, len(df[op_string][ind])):
            
            if df[op_string][ind][i] in df[manual_tagged_string][ind]:
                TP = TP + 1
                
            else:
                pass
                
        tp.append(TP)

    return tp

In [48]:
fp_entry =[]
# fn_entry = []

In [49]:
# import numpy as np
# list_1 = ["a", "b", "c", "d", "e"]
# list_2 = ["a", "f", "c", "m"] 
# # yields the elements in `list_2` that are NOT in `list_1`
# main_list = np.setdiff1d(list_2,list_1)
# yields the elements in `list_2` that are NOT in `list_1`



def get_FP(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        main_list = np.setdiff1d(df[op_string][ind],df[manual_tagged_string][ind])
        fp.append(len(main_list))
        fp_entry.append(main_list)
        
        
    return fp, fp_entry

In [50]:
# ORIGINAL
# def get_FN(df,manual_tagged_string, op_string):
    
#     for ind in df.index:
        
#         main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
#         fn.append(len(main_list))
        
#     return fn

In [51]:
def get_FN(df,manual_tagged_string, op_string):
    
    for ind in df.index:
        
        main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
        if df[manual_tagged_string][ind]=='[]':
            fn.append(0)
        else:
            fn.append(len(main_list))
        
    return fn

In [52]:
# entry like Empty|list of technology output


In [53]:
# def get_FN(df,manual_tagged_string, op_string):
    
#     for ind in df.index:
        
#         main_list = np.setdiff1d(df[manual_tagged_string][ind], df[op_string][ind])
#         if (manual_tagged_string[ind] == "[]"or manual_tagged_string[ind] == ""):
#             fn.append(0)
#         else:
#             fn.append(len(main_list))
        
#     return fn

In [54]:
# function call
tp = get_TP(df_cm, "Eng_Label", col_name_op)
fp, fp_entry_names = get_FP(df_cm, "Eng_Label", col_name_op)
fn = get_FN(df_cm, "Eng_Label", col_name_op)

# tp = get_TP(df_cm, "Tech_Label", col_name_op)
# fp, fp_entry_names = get_FP(df_cm, "Tech_Label", col_name_op)
# fn = get_FN(df_cm, "Tech_Label", col_name_op)


# tp = get_TP(df_cm, "Technology", col_name_op)
# fp = get_FP(df_cm, "Technology", col_name_op)
# fn = get_FN(df_cm, "Technology", col_name_op)

In [55]:
# tp

In [56]:
print(len(tp), len(fp), len(fn))

391 391 391


In [57]:
len(fp_entry_names)

391

In [58]:
type(fp_entry_names)

list

In [59]:
dict_fp = {}

In [60]:
for i in range(0, len(fp_entry_names)):
    fp_entry_names[i] = fp_entry_names[i].tolist()
    dict_fp[i] = fp_entry_names[i]

In [61]:
# dict_fp

In [62]:
# for i in range(0, len(fn_entry_names)):
# #     fn_entry_names[i] = fn_entry_names[i].tolist()
#     dict_fn[i] = fn_entry_names[i]

In [63]:
# dict_fn

In [64]:
# len(dict_fp)

In [65]:
# df_fp = pd.DataFrame(dict_fp.items(), columns=['index', 'FP_entries_string'])
# df_fn = pd.DataFrame(dict_fp.items(), columns=['index', 'FN_entries_combined'])

In [66]:
# len(df_fp)

In [67]:
# df_fp_fn = pd.concat([df_fp, df_cm], axis = 1)

In [68]:
# df_fp_fn.head()

In [69]:
# df_fp_fn.to_csv("FP_combined.csv", index = False)

In [70]:
# creating a dataframe
df_matrix = pd.DataFrame(list(zip(tp, fp, fn)), 
               columns =['TP', 'FP', "FN"])
df_matrix.head(-1)

,TP,FP,FN
0,0,0,0
1,3,0,0
2,0,0,0
3,0,0,0
4,2,1,0
5,0,0,0
6,1,0,0
7,0,0,0
8,3,0,0
9,0,0,0


In [71]:
def get_precision(tp,fp):
    
    if tp+fp ==0:
        precision = 0
    else:
        precision = tp/(tp+fp)
    
    return precision

In [72]:
def get_recall(tp,fn):
    
    if tp+fn ==0:
        recall = 0
    else:
        recall = tp/(tp+fn)
    
    return recall

In [73]:
def get_f_score(precision, recall):
    if precision+recall == 0:
        f_score = 0
    else:
        f_score = 2*(precision*recall)/(precision+recall)
        
    return f_score

In [74]:
df_matrix['Precision'] = df_matrix.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

In [75]:
df_matrix["Recall"] = df_matrix.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

In [76]:
df_matrix["f_score"] = df_matrix.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)

In [77]:
df_matrix.head()

,TP,FP,FN,Precision,Recall,f_score
0,0,0,0,0.000000,0.0,0.0
1,3,0,0,1.000000,1.0,1.0
2,0,0,0,0.000000,0.0,0.0
3,0,0,0,0.000000,0.0,0.0
4,2,1,0,0.666667,1.0,0.8


In [78]:
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_ner_en_large_flair_100_tagged.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_spacy_adding_more_tags.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_String_Match_non_context.csv", index = False)
# df_matrix.to_csv("189_BERT_tags_SPACY_TEST_DATA_M1_12_Jun_21.csv", index = False)

# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_lg__moreTAGS_tagged_100_more_ENTS.csv", index = False)
# df_matrix.to_csv("Train_Data/Accuracy_Data/Dataset_to_run_on_all/Output/Accuracy_Sheet_lg_100_simple_spacy.csv", index = False)


In [79]:
final_data = df_cm.join(df_matrix)
final_data.to_csv("ACC-ENG_BERT_pred_ENG_SUM_EXP.csv")
# final_data.to_csv("1190_spacy_pred_tech_match.csv")